In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
inception_model = applications.InceptionV3(weights = 'imagenet',
                                           input_shape = (150, 150, 3),
                                           include_top = False)

In [3]:
inception_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                       

In [4]:
for layer in inception_model.layers:
    layer.trainable = False

In [5]:
def train_test_generator(train_dir, test_dir):
    train_datagen = ImageDataGenerator(rescale = 1./255,
                                       width_shift_range = 0.2,
                                       height_shift_range = 0.2,
                                       rotation_range = 40,
                                       shear_range = 0.2,
                                       zoom_range = 0.2,
                                       horizontal_flip = True,
                                       fill_mode = 'nearest')
    
    train_generator = train_datagen.flow_from_directory(train_dir,
                                                        target_size = (150, 150),
                                                        class_mode = 'binary',
                                                        batch_size = 32)
    
    test_datagen = ImageDataGenerator(rescale = 1./255)
    
    test_generator = test_datagen.flow_from_directory(test_dir,
                                                      target_size = (150, 150),
                                                      class_mode = 'binary',
                                                      batch_size = 32)
    
    return train_generator, test_generator

In [6]:
training_dir = '/Users/vuhan/Desktop/dog-vs-cat-classification/data/training/'
testing_dir = '/Users/vuhan/Desktop/dog-vs-cat-classification/data/testing/'

train_generator, test_generator = train_test_generator(training_dir, testing_dir)

Found 22500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [7]:
last_desired_layer = inception_model.get_layer('mixed7')
last_output = last_desired_layer.output

In [8]:
def create_model(pre_trained_model, last_output):
    x = tf.keras.layers.Flatten()(last_output)
    x = tf.keras.layers.Dense(1024, activation = 'relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(256, activation = 'relu')(x)
    x = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)
    
    model = tf.keras.Model(inputs = pre_trained_model.input, outputs = x)
    
    model.compile(loss = 'binary_crossentropy',
                  optimizer = 'adam',
                  metrics = ['accuracy'])
    return model

In [9]:
model = create_model(inception_model, last_output)

In [10]:
total_params = model.count_params()
num_trainable_params = sum([w.shape.num_elements() for w in model.trainable_weights])

print(f"There are {total_params:,} total parameters in this model.")
print(f"There are {num_trainable_params:,} trainable parameters in this model.")

There are 47,774,113 total parameters in this model.
There are 38,798,849 trainable parameters in this model.


In [11]:
import time
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor = 'val_loss',
                           min_delta = 0.001,
                           patience = 5)
t1 = time.time()
history = model.fit(train_generator,
                    validation_data = test_generator,
                    epochs = 100,
                    callbacks = [early_stop])
t2 = time.time()

print('\nTraining model took {:.2f} minutes.'.format((t2 - t1) / 60))

Epoch 1/100
704/704 [==============================] - 345s 487ms/step - loss: 0.2358 - accuracy: 0.9116 - val_loss: 0.0854 - val_accuracy: 0.9628
Epoch 2/100
704/704 [==============================] - 352s 500ms/step - loss: 0.1550 - accuracy: 0.9351 - val_loss: 0.0805 - val_accuracy: 0.9648
Epoch 3/100
704/704 [==============================] - 359s 510ms/step - loss: 0.1366 - accuracy: 0.9424 - val_loss: 0.0773 - val_accuracy: 0.9668
Epoch 4/100
704/704 [==============================] - 354s 502ms/step - loss: 0.1348 - accuracy: 0.9459 - val_loss: 0.0821 - val_accuracy: 0.9656
Epoch 5/100
704/704 [==============================] - 350s 497ms/step - loss: 0.1314 - accuracy: 0.9476 - val_loss: 0.0903 - val_accuracy: 0.9636
Epoch 6/100
704/704 [==============================] - 351s 497ms/step - loss: 0.1273 - accuracy: 0.9490 - val_loss: 0.0801 - val_accuracy: 0.9680
Epoch 7/100
704/704 [==============================] - 353s 501ms/step - loss: 0.1292 - accuracy: 0.9458 - val_loss: 0

In [12]:
model.save('Dog_Cat_classification.h5')
print('Saved!!!')

Saved!!!


In [3]:
from tensorflow.keras.models import load_model

dc_model = load_model('Dog_Cat_classification.h5')

In [18]:
import os

os.getcwd()

'/Users/vuhan/Desktop/dog-vs-cat-classification'

In [23]:
predict_dir = os.path.join(os.getcwd(), 'test/test')
predict_dir

'/Users/vuhan/Desktop/dog-vs-cat-classification/test/test'

In [24]:
import pandas as pd
predict_df = pd.DataFrame({'id': os.listdir(predict_dir)})

In [25]:
predict_df.head(3)

,id
0,003301.jpg
1,005770.jpg
2,001516.jpg


In [45]:
from tensorflow.keras.preprocessing.image import load_img

predict_gen = ImageDataGenerator(rescale = 1./255)

predict_generator = predict_gen.flow_from_dataframe(predict_df,
                                                    predict_dir,
                                                    x_col = 'id',
                                                    y_col = None,
                                                    class_mode = None,
                                                    target_size = (150, 150),
                                                    batch_size = 32,
                                                    shuffle = False)

Found 8000 validated image filenames.


In [46]:
img_predict = dc_model.predict(predict_generator)

In [49]:
np.argmax(img_predict, axis = -1)

array([0, 0, 0, ..., 0, 0, 0])

In [57]:
img_predict[0]

array([0.00035563], dtype=float32)

In [54]:
def cnn_predict_img(model, path_img, target_size, class_0, class_1):
    import numpy as np
    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg
    from tensorflow.keras.models import load_model
    from tensorflow.keras.preprocessing import image
    model_clf = load_model(model)
    test_img = image.load_img(path_img, target_size = target_size)
    test_img = image.img_to_array(test_img)
    test_img /= 255
    test_img = np.expand_dims(test_img, axis = 0)
    
    result = model_clf.predict(test_img)
    if result[0][0] > 0.5:
        print('{}: {}'.format(class_1, result[0][0]))
    else:
        print(class_0)
    img = mpimg.imread(path_img)
    plt.imshow(img)
    plt.show()

In [63]:
labels = []
for result in img_predict:
    if result[0] > 0.5:
        labels.append(1)
    else:
        labels.append(0)

In [65]:
predict_df['labels'] = labels

In [68]:
predict_df.sort_values(by = 'id', inplace = True)

In [72]:
predict_df.reset_index(drop = True, inplace = True)

In [74]:
predict_df['labels'].value_counts()

1    4059
0    3941
Name: labels, dtype: int64

In [76]:
predict_df.to_csv('Submission.csv')

In [77]:
a = pd.read_csv('sample_submission.csv')
a

,id,labels
0,000000.jpg,1
1,000001.jpg,1
2,000002.jpg,1
3,000003.jpg,1
4,000004.jpg,1
...,...,...
7995,007995.jpg,0
7996,007996.jpg,0
7997,007997.jpg,0
7998,007998.jpg,0


In [79]:
predict_df.set_index('id', inplace = True)

In [80]:
predict_df.to_csv('Submission.csv')